# Cleaning whatsapp chat message backup to generate dataset

In [9]:
import numpy as np
import pandas as pd
import re
import emoji

In [10]:
# loading the data
data_file_raw = np.array([])
with open("source_full.txt","r",encoding="utf-8") as data_file:
    i = 0
    for line in data_file:
        if i>1:
            break
        data_file_raw = np.append(data_file_raw,[line])

In [11]:
np.random.shuffle(data_file_raw)

In [12]:
def remove_noice(text):
  # removing non letter sequences
  text = re.sub(r'(?=\s|^)([^a-zA-Z]+)(?=\s|$)',"",text)
  # removing web addresses
  text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))","",text)
  # removing new lines
  text.replace("\n"," ")
  # removing new line like character sequences
  text.replace("\\n"," ")
  # replacing multiple spaces with single space
  text = re.sub(r"\s\s+"," ",text)
  # removing emojis
  text = re.sub(emoji.get_emoji_regexp(),r"",text)
  # removing non letter,number or space characters
  text = re.sub(r'[^a-zA-Z\s0-9]',"",text)
  # removing leading and tailing white spaces and returning
  return text.strip()

In [13]:
filtered_stage_1 = []
# reformat multi lines
i = -1
for index,line in enumerate(data_file_raw):
  line = line.strip()
  # removing zero length empty spaces
  line = (line.encode('ascii', 'ignore')).decode("utf-8")
  # catching leading part of whatsapp message record syntax to check if line is
  # the start of a new message or a new line of another message
  matched = re.match(r"(?:^\d+/\d+/\d+,.\d+:\d+.(?:AM|PM).-(?:.+):)(.+)",line)
  if matched: # new message
    sentence = matched.group(1)
    sentence = remove_noice(sentence)
    filtered_stage_1.append(sentence)
    i+=1
  else: # new line of previous message
    line = remove_noice(line)
    if len(line)==0:
      pass
    # new line of previous message is appended to previous message
    filtered_stage_1[i]+=line

AttributeError: module 'emoji' has no attribute 'get_emoji_regexp'

## Filtering

In [ ]:
#removing single word sentences
filtered_stage_2 = filter(lambda x: len(re.split(r'[.,:\s+]',x))>1,filtered_stage_1)

#removing duplicates
filtered_stage_3 = list(set(filtered_stage_2))


In [ ]:
df = pd.DataFrame( filtered_stage_3,columns=['Message text'])
print(df)

In [ ]:
df.to_excel("data_frame.xlsx",sheet_name="Whatsapp")

In [ ]:
with open("out.txt","w+",encoding="utf-8") as out_file:
  out_file.writelines("\n".join(filtered_stage_3))
  out_file.close()